Parsing gencode file takes ~40sec

In [129]:
import pandas as pd
from functools import reduce


class ResidueMapper:
    
    def __init__(self, GENCODEFile: str, verbose: bool =False) -> None:
        
        self.verbose = verbose

        # Reading GENCODE data and filter for CDS segments:
        gtf_columns = ['chr', 'source', 'featureType', 'start', 'end', 'score', 'strand', 'phase', 'annotation']
        df = (
            pd.read_csv(GENCODEFile, sep='\t', comment='#', header=0, names=gtf_columns)
            .query('featureType == "CDS"')
            .reset_index(drop=True)
            .drop(['source', 'score', 'phase'], axis=1)
        )

        # Parse annotations for separate columns:
        annotations_df = pd.DataFrame(
            df.annotation
            .apply(lambda features: {feature.split('=')[0]: feature.split('=')[1] for feature in features.split(';')})
            .to_list()
        )

        # Join annotations with coordinates:
        self.full_gencode = (
            df
            .drop('annotation', axis=1)
            .merge(annotations_df, left_index=True, right_index=True)

            # Remove versions:
            .assign(
                gene_id = lambda df: df.gene_id.str.replace(r'\.\d+', '', regex=True),
                transcript_id = lambda df: df.transcript_id.str.replace(r'\.\d+', '', regex=True),
                protein_id = lambda df: df.protein_id.str.replace(r'\.\d+', '', regex=True),
                exon_id = lambda df: df.exon_id.str.replace(r'\.\d+', '', regex=True),
                chr = lambda df: df.chr.str.replace(r'chr', '', regex=False),
                feature_length = lambda df: df.end - df.start
            )

            # Dropping unused columns:
            .drop([
                'ID', 'Parent', 'gene_type', 'transcript_type', 'transcript_support_level',
                'exon_number', 'level', 'hgnc_id', 'tag', 'havana_gene', 'havana_transcript',
                'ccdsid', 'ont', 'featureType', 'transcript_name'
            ], axis=1)
        )
    
    def map_position(self, translation_id: str, aminoacid_position: int) -> list:

        # Filter gencode data for relevant protein only:
        selected = (
            self.full_gencode
            .query('protein_id == @translation_id')
            .sort_values('start')
            .reset_index(drop=True)
        )

        # Extract relevant data:
        row = selected.iloc[0]
        gene_id = row['gene_id']
        chromosome = row['chr']
        transcript_id = row['transcript_id']
        gene_name = row['gene_name']
        strand = row['strand']

        # Magic:
        positions = reduce(lambda x,y: x+y,(selected.apply(lambda row: list(range(row['start'],row['end']+1)), axis = 1)),[])

        # If the gene is on the negative strand we need to flip the list:
        if strand == '-':
            positions = positions[::-1]

        # Extracting positions:
        extracted_positions = positions[(aminoacid_position-1)*3:(aminoacid_position-1)*3+3]
        return [{'chr': chromosome, 'pos': position, 'strand': strand} for position in extracted_positions]


# Source file:
gencodeFile = '/Users/dsuveges/Downloads/gencode.v40.annotation.gff3.gz'

# Map an amino acid position of a given protein:
translation_id = 'ENSP00000327251'
aminoacid_position = 115

mapper = ResidueMapper(gencodeFile)
mapper.map_position(translation_id, aminoacid_position)




[{'chr': '17', 'pos': 27787802, 'strand': '-'},
 {'chr': '17', 'pos': 27787801, 'strand': '-'},
 {'chr': '17', 'pos': 27787800, 'strand': '-'}]

In [130]:
%%bash

chrom=17
start=27787800
end=27787802

curl -s https://rest.ensembl.org/sequence/region/human/${chrom}:${start}..${end}:-1?content-type=text/plain

TGC

In [131]:
# An other example:
translation_id = 'ENSP00000267101'
aminoacid_position = 420

mapper.map_position(translation_id, aminoacid_position)



[{'chr': '12', 'pos': 56093060, 'strand': '+'},
 {'chr': '12', 'pos': 56093061, 'strand': '+'},
 {'chr': '12', 'pos': 56093062, 'strand': '+'}]

In [132]:
# An other re-run:
translation_id = 'ENSP00000327251'
aminoacid_position = 115

mapper.map_position(translation_id, aminoacid_position)



[{'chr': '17', 'pos': 27787802, 'strand': '-'},
 {'chr': '17', 'pos': 27787801, 'strand': '-'},
 {'chr': '17', 'pos': 27787800, 'strand': '-'}]

One query takes ~100ms. So 10 lookup per second. 250000 lookups ~7 hours without parallelization.